# **Sử dụng model từ file bert_model.pkl**

In [1]:
#!pip install datasets transformers

In [2]:
import pickle
import re
import pandas as pd
import numpy as np
from datasets import Dataset

from transformers import BertTokenizer,BertForSequenceClassification,TrainingArguments,Trainer

C:\Users\tungb\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\tungb\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\tungb\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Tiền xử lý dữ liệu

In [3]:
def preprocessing_data(text, tokenizer, max_word_length):
    df=pd.DataFrame.from_dict({'text' : [text]})

    df['text']=df['text'].apply(lambda x: x.lower())
    df['text']=df['text'].apply(lambda x: x.replace("#", ""))
    df['text']=df['text'].apply(lambda x: re.sub(r'@\w+\s*', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['text']=df['text'].apply(lambda x: re.sub(r'\d+', '', x))
    df['text']=df['text'].apply(lambda x : re.compile(r'<.*?>').sub(r'', x))

    df['input_ids']=df['text'].apply(lambda x: tokenizer(x, max_length=max_word_length, padding='max_length')['input_ids'])
    df=df[['input_ids']]

    return Dataset.from_pandas(df)

BERT Model

In [4]:
with open('BE/bert_model.pkl', 'rb') as f:
    data = pickle.load(f)

tokenizer = data['tokenizer']
max_word_length = data['max_word_length']
model=data['model']
args=data['args']

In [5]:
from BE.get_tweet import GetTweet
get_tweet = GetTweet("0326130017", r"project_ML_13072023")

In [6]:
trainer = Trainer(
    model=model,
    args=args, 
)

**Input đầu vào:**

Điền một link bài tweet bạn muốn kiểm tra:

In [7]:
url="https://twitter.com/ameliya_ii/status/1675368182092271621?s=20"
#url="https://twitter.com/FORIUVS/status/1676569361841287169?s=20"

In [8]:
text = get_tweet.get_text_form_tweet(url)
print(text)

The Titan submersible, which went missing with five passengers onboard earlier this week, was destroyed after suffering a "catastrophic implosion", authorities involved in the search for the vessel said on Thursday.
#OceanGate #submarinemissing #Titan #implosion #TitanicRescue


In [9]:
output_ds=preprocessing_data(text,tokenizer,max_word_length)

In [10]:
outputs=trainer.predict(output_ds)

Kết quả dự đoán:

In [11]:
print(text)
print()
if outputs.predictions.argmax(1)[0]==1:
    print("Đây là tweet có nội dung liên quan đến thảm họa.")
else:
    print("Đây là tweet có nội dung không liên quan đến thảm họa.")

The Titan submersible, which went missing with five passengers onboard earlier this week, was destroyed after suffering a "catastrophic implosion", authorities involved in the search for the vessel said on Thursday.
#OceanGate #submarinemissing #Titan #implosion #TitanicRescue

Đây là tweet có nội dung liên quan đến thảm họa.
